Общие утилиты (чтение, нормали, сохранение)

In [1]:
# !pip install open3d trimesh scipy scikit-image pymeshlab
import numpy as np
import open3d as o3d
from pathlib import Path
import pymeshlab as ml
from dataclasses import dataclass

# Вход и выход
IN_OBJ  = Path("input/Untitled.obj")   # ← поменяй на свой путь
OUT_OBJ = Path("output/repaired_surface.obj")
OUT_OBJ.parent.mkdir(parents=True, exist_ok=True)

def load_mesh_as_pcd(obj_path, sample_n=200_000):
    """Читает .obj как mesh и равномерно сэмплирует точки c нормалями."""
    mesh = o3d.io.read_triangle_mesh(str(obj_path))
    if not mesh.has_vertex_normals():
        mesh.compute_vertex_normals()
    # Сэмплируем точки с поверхности, чтобы убрать топологические артефакты
    pcd = mesh.sample_points_poisson_disk(number_of_points=sample_n)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.02, max_nn=60))
    pcd.orient_normals_consistent_tangent_plane(100)
    return pcd

def save_mesh(mesh, out_path):
    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    o3d.io.write_triangle_mesh(str(out_path), mesh, write_triangle_uvs=False)
    return out_path

In [ ]:
import open3d as o3d

def visualize_open3d_inline(obj_path):
    mesh = o3d.io.read_triangle_mesh(str(obj_path))
    mesh.compute_vertex_normals()
    # откроет WebGL-окно прямо в ячейке Lab 4 / ноутбуке
    o3d.visualization.draw([mesh])

visualize_open3d_inline(OUT_OBJ)

FEngine (64 bits) created at 0x31fc0c000 (threading is enabled)
FEngine resolved backend: OpenGL


: 

1) Poisson Surface Reconstruction (Open3D)

Когда использовать: нужен быстрый и надёжный «водонепроницаемый» меш, который сам закрывает дырки. Требуются нормали.

In [1]:
import numpy as np
import open3d as o3d
import trimesh
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)
def fast_surface_reconstruct(obj_path, out_path):
    tm = trimesh.load_mesh(str(obj_path), process=False)
    mesh = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(tm.vertices),
        triangles=o3d.utility.Vector3iVector(tm.faces)
    )
    mesh.compute_vertex_normals()

    # Сэмплируем точки
    pcd = mesh.sample_points_uniformly(number_of_points=30000)
    pcd.estimate_normals()

    # Радиусы подбираются под масштаб модели
    radii = o3d.utility.DoubleVector([0.02, 0.04, 0.08])
    recon = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, radii)
    recon.remove_duplicated_vertices()
    recon.remove_degenerate_triangles()
    recon.compute_vertex_normals()
    o3d.io.write_triangle_mesh(str(out_path), recon)
    print(f"✅ Ball-Pivoting mesh saved to {out_path}")
    return recon

mesh = fast_surface_reconstruct(IN_OBJ, OUT_OBJ)

visualize_obj_inline(OUT_OBJ)


NameError: name 'IN_OBJ' is not defined


⸻

2) MLS (Moving Least Squares) через MeshLab/pyMeshLab

Когда использовать: нужно именно проекция/сглаживание MLS и затем «залатать» отверстия на триангуляции. PyMeshLab — это API к фильтрам MeshLab.

In [9]:
!pip install pymeshlab 
import open3d as o3d
import trimesh
import numpy as np
from pathlib import Path

def repair_surface(obj_path, out_path, method="mls"):
    tm = trimesh.load_mesh(str(obj_path), process=False)
    mesh = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(tm.vertices),
        triangles=o3d.utility.Vector3iVector(tm.faces)
    )
    mesh.compute_vertex_normals()

    # Cэмплируем точки
    pcd = mesh.sample_points_uniformly(number_of_points=50_000)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=60))

    if method == "poisson":
        print("🔧 Running Poisson reconstruction...")
        mesh, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
            pcd, depth=8, linear_fit=True
        )
    elif method == "ball":
        print("🔧 Running Ball-Pivoting reconstruction...")
        radii = o3d.utility.DoubleVector([0.02, 0.04, 0.08])
        mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, radii)
    else:
        raise ValueError("method must be 'poisson' or 'ball'")

    # Очистка и сглаживание
    mesh.remove_degenerate_triangles()
    mesh.remove_duplicated_triangles()
    mesh.remove_duplicated_vertices()
    mesh.remove_non_manifold_edges()
    mesh = mesh.filter_smooth_simple(number_of_iterations=3)
    mesh.compute_vertex_normals()

    o3d.io.write_triangle_mesh(str(out_path), mesh)
    print(f"✅ Saved to {out_path}")
    return mesh

# mesh = repair_surface(IN_OBJ, OUT_OBJ, method="ball")  
visualize_obj_inline(OUT_OBJ)


/Users/dmitry057/Projects/DeepL/archi-ve/FreeCAD/.conda/ml/lib/python3.10/site-packages/traittypes/traittypes.py:98: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  warnings.warn(
/Users/dmitry057/Projects/DeepL/archi-ve/FreeCAD/.conda/ml/lib/python3.10/site-packages/pythreejs/traits.py:257: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '
/Users/dmitry057/Projects/DeepL/archi-ve/FreeCAD/.conda/ml/lib/python3.10/site-packages/traittypes/traittypes.py:98: UserWarning: Given trait value dtype "uint32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


Renderer(background='white', camera=PerspectiveCamera(position=(3.0, 3.0, 3.0), projectionMatrix=(1.0, 0.0, 0.…

Примечание: Если хочешь именно MLS-проекцию по облаку, лучше:
	1.	из Open3D получить pcd → ms.load_new_mesh(pcd) (через временный PLY),
	2.	ms.apply_filter('mls_projection', ...),
	3.	затем ball_pivoting или poisson в MeshLab для триангуляции.

### 3) RBF-implicit surface: глобальная гладкая функция и Marching Cubes

Когда использовать: нужна управляемая гладкость и заполнение отверстий «по физике» непрерывной функции.
Идея: аппроксимируем signed distance к поверхности RBF-функцией, строим изосurface f(x)=0.

In [ ]:
# Этот код был предназначен для standalone-скрипта, но конфликтует с основной функцией.
# Закомментирован для избежания конфликтов. Используйте функцию из Cell 5.

# # repair_mls.py
# import pymeshlab as ml
# import argparse
# from pathlib import Path
# 
# def mls_fill_via_pymeshlab_standalone(obj_path, out_path,
#                            mls_radius=0.02, mls_smoothing_steps=2,
#                            hole_area_max=1e-3, laplacian_steps=10):
#     """
#     1. Выполняет MLS-сглаживание (или Poisson reconstruction)
#     2. Заполняет отверстия
#     3. Применяет лапласово сглаживание
#     """
#     ms = ml.MeshSet()
#     ms.load_new_mesh(str(obj_path))
# 
#     # --- Если вход — point cloud ---
#     # ms.apply_filter('mls_projection', radius=mls_radius, stepsmoothnum=mls_smoothing_steps)
# 
#     # --- Если вход — mesh (обычно OBJ) ---
#     ms.apply_filter('compute_normal_for_point_clouds', k=16)
#     ms.apply_filter('surface_reconstruction_screened_poisson', depth=9)
# 
#     # Заполняем дыры (по площади)
#     ms.apply_filter('close_holes', maxholesize=hole_area_max)
# 
#     # Сглаживаем края
#     ms.apply_filter('laplacian_smooth', steps=laplacian_steps, cotangentweight=True)
# 
#     ms.save_current_mesh(str(out_path))
#     print(f"✅ Saved smoothed mesh to: {out_path}")
# 
# def main():
#     parser = argparse.ArgumentParser(description="Fill and smooth mesh using MLS/Poisson via PyMeshLab")
#     parser.add_argument("--in_path", required=True, help="Path to input .obj file")
#     parser.add_argument("--out_path", required=True, help="Path to save output .obj file")
#     parser.add_argument("--radius", type=float, default=0.02)
#     parser.add_argument("--smooth_steps", type=int, default=2)
#     parser.add_argument("--hole_area", type=float, default=1e-3)
#     parser.add_argument("--laplacian_steps", type=int, default=10)
#     args = parser.parse_args()
# 
#     mls_fill_via_pymeshlab_standalone(
#         obj_path=args.in_path,
#         out_path=args.out_path,
#         mls_radius=args.radius,
#         mls_smoothing_steps=args.smooth_steps,
#         hole_area_max=args.hole_area,
#         laplacian_steps=args.laplacian_steps
#     )


Проверка результата в Open3D


In [ ]:
# Короткая проверка результата в Open3D
import open3d as o3d

mesh = o3d.io.read_triangle_mesh(str(OUT_OBJ))
mesh.remove_duplicated_vertices()
mesh.remove_duplicated_triangles()
mesh.remove_degenerate_triangles()
mesh.remove_non_manifold_edges()
mesh.compute_vertex_normals()

print(mesh)
print("Vertices:", len(mesh.vertices))
print("Faces   :", len(mesh.triangles))


Батч-режим по директории


In [ ]:
def batch_repair(in_dir: Path, out_dir: Path, cfg: MLSConfig, glob: str = "*.obj"):
    """
    Обрабатывает все файлы из директории по маске glob.
    
    Args:
        in_dir: Входная директория с OBJ файлами
        out_dir: Выходная директория для результатов
        cfg: Конфигурация MLSConfig
        glob: Маска файлов (по умолчанию "*.obj")
    """
    out_dir.mkdir(parents=True, exist_ok=True)
    for p in sorted(in_dir.glob(glob)):
        out_p = out_dir / (p.stem + "_repaired.obj")
        try:
            mls_fill_via_pymeshlab(p, out_p, cfg)
            print(f"✓ {p.name} → {out_p.name}")
        except Exception as e:
            print(f"✗ {p.name}: {e}")

# Пример использования:
# cfg = MLSConfig(mode="poisson")  # или mode="mls"
# batch_repair(Path("input"), Path("output"), cfg)
